In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE

from imblearn.pipeline import pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [41]:
url = "https://raw.githubusercontent.com/digipodium/Datasets/main/classfication/bank_data.csv"
df = pd.read_csv(url,index_col=0)
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
df.drop(columns=['CustomerId', 'Surname'], inplace=True)
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [43]:
print("Data Null  value check ")
df.isnull().sum()

Data Null  value check 


CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [44]:
X = df.drop(columns='Exited')
y = df['Exited']

In [45]:
X.select_dtypes('object').columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Geography', 'Gender'], dtype='object')>

In [46]:
X.select_dtypes(np.number).columns.tolist()

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

### Pipeline creation

In [47]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_transformer = OneHotEncoder(handle_unknown="ignore")    

In [48]:
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, X.select_dtypes(np.number).columns.tolist()),
        ("category", categorical_transformer, X.select_dtypes('object').columns.tolist()),
    ]
)

In [49]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor),
     
    ("classifier", RandomForestClassifier(n_estimators= 5))]
)


In [50]:
clf.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('numeric',
                                    Pipeline(steps=[('imputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('scaler', StandardScaler())]),
                                    ['CreditScore', 'Age', 'Tenure', 'Balance',
                                     'NumOfProducts', 'HasCrCard',
                                     'IsActiveMember', 'EstimatedSalary']),
                                   ('category',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    ['Geography', 'Gender'])])),
  ('classifier', RandomForestClassifier(n_estimators=5))],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('numeric',
                                  Pipeline(steps=[('imputer',
                                                   SimpleIm

In [51]:
param_grid = {
    'classifier__min_samples_split':[2, 3, 4]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

In [53]:
grid_search.fit(X_train, y_train)
print('Best params:')
print(grid_search.best_params_)


Best params:
{'classifier__min_samples_split': 4}


In [54]:
print(f"Interval CVscore: {grid_search.best_score_:.3f}")

Interval CVscore: 0.838


In [55]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1468  127]
 [ 198  207]]
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1595
           1       0.62      0.51      0.56       405

    accuracy                           0.84      2000
   macro avg       0.75      0.72      0.73      2000
weighted avg       0.83      0.84      0.83      2000



In [56]:
from joblib import dump
dump(clf, 'model/churn_model_v1.jb')


['model/churn_model_v1.jb']

In [57]:
## App sklearn

X.iloc[0]      # sample

CreditScore              619
Geography             France
Gender                Female
Age                       42
Tenure                     2
Balance                  0.0
NumOfProducts              1
HasCrCard                  1
IsActiveMember             1
EstimatedSalary    101348.88
Name: 1, dtype: object

In [58]:
X.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)